In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import sys
sys.executable

'/home/ggg/miniconda3/envs/nebula/bin/python'

In [3]:
!export CUDA_VISIBLE_DEVICES=0,1   # 双卡4090并行

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [4]:
from llama_index.llms.vllm import Vllm   # 用vllm加速本地模型
import numpy as np

llm = Vllm(
    model="/home/ggg/.cache/huggingface/hub/glm-4-9b-chat",
    dtype="float16",
    tensor_parallel_size=2,
    temperature=0,
    max_new_tokens=100,
    vllm_kwargs={
        "swap_space": 1,
        "gpu_memory_utilization": 0.8,
        "max_model_len": 8192,
    },
)

WARNING 12-01 19:19:14 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 12-01 19:19:24 config.py:1020] Defaulting to use mp for distributed inference
INFO 12-01 19:19:24 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='/home/ggg/.cache/huggingface/hub/glm-4-9b-chat', speculative_config=None, tokenizer='/home/ggg/.cache/huggingface/hub/glm-4-9b-chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_mo

Loading safetensors checkpoint shards:   0% Completed | 0/10 [00:00<?, ?it/s]


(VllmWorkerProcess pid=4092930) INFO 12-01 19:19:51 model_runner.py:1077] Loading model weights took 8.8289 GB
INFO 12-01 19:19:53 model_runner.py:1077] Loading model weights took 8.8289 GB
(VllmWorkerProcess pid=4092930) INFO 12-01 19:19:57 worker.py:232] Memory profiling results: total_gpu_memory=23.99GiB initial_memory_usage=10.56GiB peak_torch_memory=9.46GiB memory_usage_post_profile=10.62GiB non_torch_memory=1.79GiB kv_cache_size=7.94GiB gpu_memory_utilization=0.80
INFO 12-01 19:19:58 worker.py:232] Memory profiling results: total_gpu_memory=23.99GiB initial_memory_usage=10.56GiB peak_torch_memory=10.27GiB memory_usage_post_profile=10.62GiB non_torch_memory=1.79GiB kv_cache_size=7.13GiB gpu_memory_utilization=0.80
INFO 12-01 19:19:58 distributed_gpu_executor.py:57] # GPU blocks: 23369, # CPU blocks: 3276
INFO 12-01 19:19:58 distributed_gpu_executor.py:61] Maximum concurrency for 8192 tokens per request: 45.64x
(VllmWorkerProcess pid=4092930) INFO 12-01 19:19:59 model_runner.py:140

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_args = {
    'model_name': '/home/ggg/.cache/huggingface/hub/text2vec-large-chinese', 
    'max_length': 512, 
    'embed_batch_size': 32, 
    'device': 'cuda'
    }
embed_model = HuggingFaceEmbedding(**embed_args)

No sentence-transformers model found with name /home/ggg/.cache/huggingface/hub/text2vec-large-chinese. Creating a new one with MEAN pooling.


In [6]:
!nvidia-smi

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Sun Dec  1 19:21:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 552.44       CUDA Version: 12.4     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:3B:00.0  On |                  Off |
| 39%   36C    P8              19W / 450W |  21536MiB / 24564MiB |      0%      Default |
|                                         |                      |                  

In [7]:
from llama_index.core import Settings
# 把加载的模型添加到环境设置中，定义全局模型

Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
import os
import json
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

from IPython.display import Markdown, display

In [9]:
documents = SimpleDirectoryReader("./datasets").load_data()

In [10]:
%load_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula
%ngql CREATE SPACE IF NOT EXISTS test(vid_type=FIXED_STRING(256));

[OK] Connection Pool Created


""


In [13]:
%ngql USE test;

""


In [14]:
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore

# 加载刚刚创建的知识图谱，如果你之前创建过，把名称改一下
graph_store = NebulaPropertyGraphStore(
    space="test", overwrite=True   # 数据库是test，这里可能有延迟，创建数据库后需要等个10s左右再连接
)

In [15]:
from llama_index.core.vector_stores.simple import SimpleVectorStore

vec_store = SimpleVectorStore()

In [16]:
%ngql ALTER TAG Props__ ADD (page_label string DEFAULT 'EMPTY');
%ngql ALTER EDGE Relation__ ADD (page_label string NULL default NULL);

""


In [17]:
from llama_index.core.indices.property_graph import PropertyGraphIndex
from llama_index.core.storage.storage_context import StorageContext

index = PropertyGraphIndex.from_documents(
    documents,
    property_graph_store=graph_store,
    vector_store=vec_store,
    show_progress=True,
)

index.storage_context.vector_store.persist("./datasets/vec_store/nebula_vec_store.json")

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.72s/it, est. speed input: 110.14 toks/s, output: 26.93 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.55s/it, est. speed input: 136.32 toks/s, output: 28.22 toks/s]

Generating embeddings: 100%|██████████| 2/2 [00:04<00:00,  2.13s/it]


In [18]:
%ngql SHOW TAGS

,Name
0,Chunk__
1,Entity__
2,Node__
3,Props__


In [19]:
%ngql SHOW EDGES

,Name
0,Relation__
1,__meta__node_label__
2,__meta__rel_label__


In [20]:
%ngql MATCH p=(v:Entity__)-[r]->(t:Entity__) RETURN v.Entity__.name AS src, r.label AS relation, t.Entity__.name AS dest LIMIT 15;

,src,relation,dest
0,华东理工大学,评选,优秀学生
1,华东理工大学,评选,优秀学生干部
2,华东理工大学,评选,先进集体
3,华东理工大学,活动,全校范围内的本科生先进个人和先进集体评选活动
4,华东理工大学,目的,培养和选树全面发展的人才
5,华东理工大学,目的,引导和建设争创先进的优良学风班集体
6,华东理工大学,目的,服务中国式现代化建设
7,1. 先进个人,Application,Student
8,基本要求,Includes,关心集体
9,基本要求,Includes,勤奋学习所学专业


In [21]:
%ngql MATCH p=(v:Entity__)-[r]->(t:Entity__) RETURN p LIMIT 1000;

,p
0,"(""华东理工大学"" :Props__{_node_content: __NULL__, _node_type: __NULL__, creation_date: ""2024-12-01"", doc_id: __NULL__, document_id: __NULL__, file_name: ""test.pdf"", file_path: ""/home/ggg/project/rag+kg-nebula/datasets/test.pdf"", file_size: 225015, file_type: ""application/pdf"", last_modified_date: ""2024-12-01"", page_label: ""1"", ref_doc_id: __NULL__, triplet_source_id: ""85a1d2a7-4e55-429c-960e-f1e81411b5e4""} :Node__{label: ""entity""} :Entity__{name: ""华东理工大学""})-[:Relation__@0{label: ""评选"", triplet_source_id: ""85a1d2a7-4e55-429c-960e-f1e81411b5e4"", file_path: ""/home/ggg/project/rag+kg-nebula/datasets/test.pdf"", file_name: ""test.pdf"", file_type: ""application/pdf"", file_size: 225015, _node_type: __NULL__, creation_date: ""2024-12-01"", _node_content: __NULL__, last_modified_date: ""2024-12-01"", document_id: __NULL__, doc_id: __NULL__, ref_doc_id: __NULL__, page_label: ""1""}]->(""优秀学生"" :Props__{_node_content: __NULL__, _node_type: __NULL__, creation_date: ""2024-12-01"", doc_id: __NULL__, document_id: __NULL__, file_name: ""test.pdf"", file_path: ""/home/ggg/project/rag+kg-nebula/datasets/test.pdf"", file_size: 225015, file_type: ""application/pdf"", last_modified_date: ""2024-12-01"", page_label: ""1"", ref_doc_id: __NULL__, triplet_source_id: ""85a1d2a7-4e55-429c-960e-f1e81411b5e4""} :Node__{label: ""entity""} :Entity__{name: ""优秀学生""})"
1,"(""华东理工大学"" :Props__{_node_content: __NULL__, _node_type: __NULL__, creation_date: ""2024-12-01"", doc_id: __NULL__, document_id: __NULL__, file_name: ""test.pdf"", file_path: ""/home/ggg/project/rag+kg-nebula/datasets/test.pdf"", file_size: 225015, file_type: ""application/pdf"", last_modified_date: ""2024-12-01"", page_label: ""1"", ref_doc_id: __NULL__, triplet_source_id: ""85a1d2a7-4e55-429c-960e-f1e81411b5e4""} :Node__{label: ""entity""} :Entity__{name: ""华东理工大学""})-[:Relation__@0{label: ""评选"", triplet_source_id: ""85a1d2a7-4e55-429c-960e-f1e81411b5e4"", file_path: ""/home/ggg/project/rag+kg-nebula/datasets/test.pdf"", file_name: ""test.pdf"", file_type: ""application/pdf"", file_size: 225015, _node_type: __NULL__, creation_date: ""2024-12-01"", _node_content: __NULL__, last_modified_date: ""2024-12-01"", document_id: __NULL__, doc_id: __NULL__, ref_doc_id: __NULL__, page_label: ""1""}]->(""优秀学生干部"" :Props__{_node_content: __NULL__, _node_type: __NULL__, creation_date: ""2024-12-01"", doc_id: __NULL__, document_id: __NULL__, file_name: ""test.pdf"", file_path: ""/home/ggg/project/rag+kg-nebula/datasets/test.pdf"", file_size: 225015, file_type: ""application/pdf"", last_modified_date: ""2024-12-01"", page_label: ""1"", ref_doc_id: __NULL__, triplet_source_id: ""85a1d2a7-4e55-429c-960e-f1e81411b5e4""} :Node__{label: ""entity""} :Entity__{name: ""优秀学生干部""})"
2,"(""华东理工大学"" :Props__{_node_content: __NULL__, _node_type: __NULL__, creation_date: ""2024-12-01"", doc_id: __NULL__, document_id: __NULL__, file_name: ""test.pdf"", file_path: ""/home/ggg/project/rag+kg-nebula/datasets/test.pdf"", file_size: 225015, file_type: ""application/pdf"", last_modified_date: ""2024-12-01"", page_label: ""1"", ref_doc_id: __NULL__, triplet_source_id: ""85a1d2a7-4e55-429c-960e-f1e81411b5e4""} :Node__{label: ""entity""} :Entity__{name: ""华东理工大学""})-[:Relation__@0{label: ""评选"", triplet_source_id: ""85a1d2a7-4e55-429c-960e-f1e81411b5e4"", file_path: ""/home/ggg/project/rag+kg-nebula/datasets/test.pdf"", file_name: ""test.pdf"", file_type: ""application/pdf"", file_size: 225015, _node_type: __NULL__, creation_date: ""2024-12-01"", _node_content: __NULL__, last_modified_date: ""2024-12-01"", document_id: __NULL__, doc_id: __NULL__, ref_doc_id: __NULL__, page_label: ""1""}]->(""先进集体"" :Props__{_node_content: __NULL__, _node_type: __NULL__, creation_date: ""2024-12-01"", doc_id: __NULL__, document_id: __NULL__, file_name: ""test.pdf"", file_path: ""/home/ggg/project/rag+kg-nebula/datasets/test.pdf"", file_size: 225015, file_type: ""application/pdf"", last_modified_date: ""2024-12-01"", p

In [22]:
%ng_draw

<class 'pyvis.network.Network'> |N|=30 |E|=22

In [23]:
retriever = index.as_retriever(
    include_text=False,  # 返回原文，设置为True，如果要让知识图谱根据问题返回节点，设置为False
)

nodes = retriever.retrieve("优秀学生怎么申请？")

for node in nodes:
    print(node.text)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.11s/it, est. speed input: 19.95 toks/s, output: 24.33 toks/s]


优秀学生 -> 评选 -> 华东理工大学
1. 优秀学生 -> Reward -> 200元
全校范围内的本科生先进个人和先进集体评选活动 -> 活动 -> 华东理工大学
引导和建设争创先进的优良学风班集体 -> 目的 -> 华东理工大学


In [24]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query("优秀学生怎么申请？")

print(str(response))

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.04s/it, est. speed input: 288.42 toks/s, output: 24.74 toks/s]

优秀学生可以通过学工系统在线申请。具体步骤包括：1. 下载填写评审表模板，上传表格附件在线提交；2. 辅导员审核，在线填写意见并提交学院学生工作负责人；3. 审批通过后提交学工部并在学院范围内公示。申请时间为2024年10月15日至19日。请注意，同一学年只能选择一项申请。如有特殊情况，可线下提交材料及汇总表格。此外，辅导员需确保审核
